In [1]:
import numpy as np
import pandas as pd
from sklearn import svm, model_selection, preprocessing, pipeline, impute, feature_selection
from sklearn.experimental import enable_iterative_imputer
from itertools import chain, combinations, product
import math
import time

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=UserWarning)

In [2]:
data = pd.read_csv("insurance.csv")
data

,GoodStudent,Age,SocioEcon,RiskAversion,VehicleYear,ThisCarDam,RuggedAuto,Accident,MakeModel,DrivQuality,...,HomeBase,AntiTheft,PropCost,OtherCarCost,OtherCar,MedCost,Cushioning,Airbag,ILiCost,DrivHist
0,False,Adult,Prole,Adventurous,Older,Moderate,EggShell,Mild,Economy,Poor,...,City,False,TenThou,Thousand,True,Thousand,Poor,False,Thousand,Many
1,False,Senior,Prole,Cautious,Current,None,Football,None,Economy,Normal,...,City,True,Thousand,Thousand,True,Thousand,Good,True,Thousand,Zero
2,False,Senior,UpperMiddle,Psychopath,Current,None,Football,None,FamilySedan,Excellent,...,City,False,Thousand,Thousand,False,Thousand,Good,True,Thousand,One
3,False,Adolescent,Middle,Normal,Older,None,EggShell,None,Economy,Normal,...,Suburb,False,Thousand,Thousand,True,Thousand,Fair,False,Thousand,Zero
4,False,Adolescent,Prole,Normal,Older,Moderate,Football,Moderate,Economy,Poor,...,City,False,TenThou,Thousand,False,Thousand,Fair,False,Thousand,Many
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,Adult,Prole,Adventurous,Older,Mild,Football,Mild,Economy,Poor,...,City,False,Thousand,Thousand,True,Thousand,Fair,False,Thousand,Many
19996,False,Adult,Middle,Normal,Older,None,Tank,None,FamilySedan,Normal,...,Suburb,False,Thousand,Thousand,True,Thousand,Good,False,Thousand,Zero
19997,False,Senior,UpperMiddle,Normal,Current,None,Football,None,Luxury,Excellent,...,Secure,True,TenThou,Thousand,False,Thousand,Excellent,True,Thousand,Zero
19998,False,Adult,Middle,Normal,Older,None,Football,None,FamilySedan,Excellent,...,Suburb,False,Thousand,Thousand,True,Thousand,Good,True,Thousand,Zero


In [3]:
#X = data[data.columns.drop(["RiskAversion", "Accident"])]
#y = data[["RiskAversion", "Accident"]]

X = data[data.columns.drop("Accident")]
y = data["Accident"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

In [164]:
le_X = preprocessing.OrdinalEncoder()
le_y = preprocessing.LabelEncoder()

X_train_en = le_X.fit_transform(X_train)
y_train_en = le_y.fit_transform(y_train)

In [165]:
clf = svm.SVC()
clf.fit(X_train_en, y_train_en)

SVC()

In [166]:
clf.predict(le_X.transform(X_test))

array([0, 2, 2, ..., 3, 2, 2])

In [167]:
# train an imputer 
imp = impute.SimpleImputer(missing_values=np.nan, strategy = "most_frequent")
#imp = impute.IterativeImputer(missing_values=np.nan, initial_strategy= "most_frequent")
imp.fit(X_train)

SimpleImputer(strategy='most_frequent')

In [8]:
prueba = X_test.head(1).to_numpy()
prueba[0][10:26] = np.nan
imp.transform(prueba)

array([[False, 'Adolescent', 'Prole', 'Adventurous', 'Older', 'Mild',
        'EggShell', 'Economy', 'Poor', 'TwentyThou', False, 'Normal',
        False, 'Thousand', False, 'FiveThou', 'City', False, 'Thousand',
        'Thousand', True, 'Thousand', 'Poor', False, 'Thousand', 'Zero']],
      dtype=object)

In [9]:
X_test.iloc[[1]]

,GoodStudent,Age,SocioEcon,RiskAversion,VehicleYear,ThisCarDam,RuggedAuto,MakeModel,DrivQuality,Mileage,...,HomeBase,AntiTheft,PropCost,OtherCarCost,OtherCar,MedCost,Cushioning,Airbag,ILiCost,DrivHist
17763,False,Adult,Middle,Normal,Older,None,Football,FamilySedan,Excellent,FiftyThou,...,Secure,False,Thousand,Thousand,True,Thousand,Fair,False,Thousand,Zero


In [10]:
X_observed = X.columns[0:20]
X_unobserved = [x for x in X if x not in X_observed]

In [11]:
def powerset(iterable):
    s = list(iterable)
    tmp = list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
    return tmp

In [12]:
S = powerset(X_unobserved)
S.pop(0)

()

In [13]:
S[10][1]

'DrivHist'

In [14]:
def omega(variable, data) :
    return list(data[variable].unique())

In [15]:
omega("Age",X)

['Adult', 'Senior', 'Adolescent']

In [16]:
# OPCIONES PARA PREDICT
# Estimar con los datos la clase más probable
# Estimar con los datos la missing data y utilizar el algoritmo
clf.predict(le_X.transform(X_test.head(1).to_numpy()))

array([0])

In [17]:
prueba = X_test.head(1).to_numpy()

In [18]:
prueba[0][10:26] = np.nan

#prueba.iloc[0,20:26] = np.nan

In [19]:
clf.predict(le_X.transform(X_test.head(1)))

array([0])

In [20]:
X_observed

Index(['GoodStudent', 'Age', 'SocioEcon', 'RiskAversion', 'VehicleYear',
       'ThisCarDam', 'RuggedAuto', 'MakeModel', 'DrivQuality', 'Mileage',
       'Antilock', 'DrivingSkill', 'SeniorTrain', 'ThisCarCost', 'Theft',
       'CarValue', 'HomeBase', 'AntiTheft', 'PropCost', 'OtherCarCost'],
      dtype='object')

In [75]:
def reduce_dataset(instance, X,y) :
    columns = X.columns
    for i,value in enumerate(instance) :
        if isinstance(value, str) or not math.isnan(value) :
            X = X[X[columns[i]] == value]
            X = X.drop(columns[i], axis=1)
    y = y[[j for j in y.index if j in X.index]]
    return X,y

In [22]:
X['Age'] == "Adult"

0         True
1        False
2        False
3        False
4        False
         ...  
19995     True
19996     True
19997    False
19998     True
19999     True
Name: Age, Length: 20000, dtype: bool

In [24]:
X_test.head(1).to_numpy()

array([[False, 'Adolescent', 'Prole', 'Adventurous', 'Older', 'Mild',
        'EggShell', 'Economy', 'Poor', 'TwentyThou', False,
        'SubStandard', False, 'Thousand', False, 'FiveThou', 'City',
        False, 'TenThou', 'Thousand', False, 'Thousand', 'Poor', False,
        'Thousand', 'Many']], dtype=object)

In [25]:
reduced = reduce_dataset(prueba[0], X)

In [26]:
reduced

,GoodStudent,Age,SocioEcon,RiskAversion,VehicleYear,ThisCarDam,RuggedAuto,MakeModel,DrivQuality,Mileage,...,HomeBase,AntiTheft,PropCost,OtherCarCost,OtherCar,MedCost,Cushioning,Airbag,ILiCost,DrivHist
433,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,TenThou,Thousand,False,Thousand,Poor,False,Thousand,Many
1900,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,TenThou,Thousand,False,Thousand,Fair,False,Thousand,One
2750,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,Rural,False,Thousand,Thousand,False,Thousand,Poor,False,Thousand,Many
3744,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,Thousand,Thousand,True,Thousand,Fair,False,Thousand,Many
4439,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,Thousand,Thousand,True,Thousand,Poor,False,Thousand,Many
4914,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,Thousand,Thousand,False,Thousand,Poor,False,Thousand,Zero
4916,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,Thousand,Thousand,False,Thousand,Fair,False,Thousand,Many
5958,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,Thousand,Thousand,True,Thousand,Fair,False,Thousand,One
6262,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,TenThou,Thousand,False,Thousand,Poor,False,Thousand,Many
6385,False,Adolescent,Prole,Adventurous,Older,Mild,EggShell,Economy,Poor,TwentyThou,...,City,False,Thousand,Thousand,False,Thousand,Poor,False,Thousand,Many


In [27]:
def omega(variables, data) :
    domains = []
    for variable in variables :
        domains.append(data[variable].unique())
    return [p for p in product(*domains)]
    

# True if R is relevant
# False if R is irrelevant
def check_r(instances, set_R, data) : 
    #Type checking
    if isinstance(instances, pd.core.frame.DataFrame) :
        instances = instances.to_numpy()
    #Array for each instance
    out_array = []
    # For each instance:
    for instance in instances :
        # Check which are the missing variables
        columns = data.columns
        unobserved_vars = []
        for i,value in enumerate(instance) :
            if not isinstance(value, str) and math.isnan(value) :
                unobserved_vars.append(columns[i])
        # Check if R in unobserved
        #print(unobserved_vars)
        for R in set_R :
            if R not in unobserved_vars :
                print("Error")
                return
        # Impute and predict the instance
        complete_instance = imp.transform([instance])
        y = clf.predict(le_X.transform(complete_instance))
        # Obtain domain of R
        omega_R = omega(set_R, data)
        # For each value assignment r in omega(R)
        out_array.append(False)
        for value_assignment_r in omega_R :
            # Fill in values
            instance_alt = instance.copy()
            for i,value in enumerate(value_assignment_r) :
                instance_alt[list(data.columns).index(set_R[i])] = value
            #print(instance)
            #print(instance_alt)
            # Imputar y predecir 
            complete_instance_alt = imp.transform([instance_alt])
            y_alt = clf.predict(le_X.transform(complete_instance_alt))
            # Comparar con prediccion de instance
            #print(y, " == ", y_alt)
            if y != y_alt :
                out_array.pop()
                out_array.append(True)
                break
    return out_array

In [28]:
np.nan == np.nan

False

In [29]:
out = check_r(prueba, X_train.columns[15:20], X_train)
out

[False]

In [30]:
for i in range(10,23) :
    print(list(range(i,i+3)),check_r(prueba, X_train.columns[i:i+3], X_train))

[10, 11, 12] [False]
[11, 12, 13] [False]
[12, 13, 14] [False]
[13, 14, 15] [False]
[14, 15, 16] [False]
[15, 16, 17] [False]
[16, 17, 18] [False]
[17, 18, 19] [False]
[18, 19, 20] [False]
[19, 20, 21] [False]
[20, 21, 22] [False]
[21, 22, 23] [False]
[22, 23, 24] [False]


In [31]:
def check_every_r(instances, data) :
    #Type checking
    if isinstance(instances, pd.core.frame.DataFrame) :
        instances = instances.to_numpy()
    #Array for each instance
    out_array = []
    # For each instance:
    for instance in instances :
        # Check which are the missing variables
        columns = data.columns
        unobserved_vars = []
        for i,value in enumerate(instance) :
            if not isinstance(value, str) and math.isnan(value) :
                unobserved_vars.append(columns[i])
        S = powerset(unobserved_vars)
        S.pop(0)
        relevant_sets = []
        irrelevant_sets = []
        for i in S :
            if check_r([instance], list(i), data)[0] :
                relevant_sets.append(i)
            else :
                irrelevant_sets.append(i)
        return relevant_sets, irrelevant_sets

In [32]:
def check_every_r_prune(instances, data) :
    #Type checking
    if isinstance(instances, pd.core.frame.DataFrame) :
        instances = instances.to_numpy()
    #Array for each instance
    out_array = []
    # For each instance:
    for instance in instances :
        # Check which are the missing variables
        columns = data.columns
        unobserved_vars = []
        for i,value in enumerate(instance) :
            if not isinstance(value, str) and math.isnan(value) :
                unobserved_vars.append(columns[i])
        S = powerset(unobserved_vars)
        S.pop(0)
        # Divide by length
        S_split = []
        size = 0
        for i in S :
            if len(i) != size:
                size = size + 1
                S_split.append([])
            S_split[-1].append(i)
        relevant_sets = []
        irrelevant_sets = []
        for i in range(0,len(S_split)) :
            for j in S_split[i] :
                # If relevant
                if check_r([instance], list(j), data)[0] :
                    relevant_sets.append(j)
                    # Apply prune
                    for k in range(i+1,len(S_split)) :
                        tmp = []
                        for l in S_split[k] :
                            if set(j).issubset(set(l)) :
                                relevant_sets.append(l)
                            else :
                                tmp.append(l)
                        S_split[k] = tmp
                # If irrelevant
                else :
                    irrelevant_sets.append(j)
        return relevant_sets, irrelevant_sets

In [174]:
prueba = X_test.head(1).to_numpy()
prueba[0][0:10] = np.nan
t0 = time.time()
relevant_sets, irrelevant_sets = check_every_r(prueba, X_train)
tf = time.time()
print("Brute force: ",tf-t0)

Brute force:  473.5526304244995


In [ ]:
t0 = time.time()
relevant_sets_p, irrelevant_sets_p = check_every_r_prune(prueba, X_train)
tf = time.time()
print("Prune: ",tf-t0)

In [ ]:
print(relevant_sets == relevant_sets_p)
print(irrelevant_sets == irrelevant_sets_p)

In [179]:
irrelevant_sets

[('GoodStudent',),
 ('Age',),
 ('SocioEcon',),
 ('RiskAversion',),
 ('VehicleYear',),
 ('RuggedAuto',),
 ('MakeModel',),
 ('DrivQuality',),
 ('Mileage',),
 ('GoodStudent', 'Age'),
 ('GoodStudent', 'SocioEcon'),
 ('GoodStudent', 'RiskAversion'),
 ('GoodStudent', 'VehicleYear'),
 ('GoodStudent', 'RuggedAuto'),
 ('GoodStudent', 'MakeModel'),
 ('GoodStudent', 'DrivQuality'),
 ('GoodStudent', 'Mileage'),
 ('Age', 'SocioEcon'),
 ('Age', 'RiskAversion'),
 ('Age', 'VehicleYear'),
 ('Age', 'RuggedAuto'),
 ('Age', 'MakeModel'),
 ('Age', 'DrivQuality'),
 ('Age', 'Mileage'),
 ('SocioEcon', 'RiskAversion'),
 ('SocioEcon', 'VehicleYear'),
 ('SocioEcon', 'RuggedAuto'),
 ('SocioEcon', 'MakeModel'),
 ('SocioEcon', 'DrivQuality'),
 ('SocioEcon', 'Mileage'),
 ('RiskAversion', 'VehicleYear'),
 ('RiskAversion', 'RuggedAuto'),
 ('RiskAversion', 'MakeModel'),
 ('RiskAversion', 'DrivQuality'),
 ('RiskAversion', 'Mileage'),
 ('VehicleYear', 'RuggedAuto'),
 ('VehicleYear', 'MakeModel'),
 ('VehicleYear', 'DrivQu

In [176]:
# TODO FSS
X_red, y_red = reduce_dataset(prueba[0], X_train, y_train)

In [186]:
X_red

,GoodStudent,Age,SocioEcon,RiskAversion,VehicleYear,ThisCarDam,RuggedAuto,MakeModel,DrivQuality,Mileage
19496,False,Adult,Prole,Adventurous,Older,None,EggShell,Economy,Poor,TwentyThou
16430,False,Adult,Prole,Adventurous,Older,None,EggShell,Economy,Poor,FiveThou
16349,False,Adult,Prole,Normal,Older,None,EggShell,SportsCar,Poor,FiftyThou
19828,False,Senior,Prole,Normal,Older,None,EggShell,Economy,Poor,TwentyThou
14271,False,Adolescent,Prole,Normal,Older,Mild,EggShell,Economy,Poor,TwentyThou
2786,False,Adult,Prole,Normal,Older,None,EggShell,Economy,Poor,TwentyThou
8660,False,Adolescent,Prole,Adventurous,Older,None,EggShell,Economy,Poor,TwentyThou
3439,False,Adult,Prole,Normal,Older,None,EggShell,Economy,Poor,Domino
1728,False,Adolescent,Prole,Normal,Older,None,EggShell,Economy,Poor,TwentyThou
19518,False,Adult,Prole,Adventurous,Older,None,EggShell,SportsCar,Poor,FiftyThou


In [ ]:
y_red

In [188]:
#sel = feature_selection.GenericUnivariateSelect()
oe = preprocessing.OrdinalEncoder()
sel = feature_selection.RFECV(svm.SVC(kernel="linear"))
sel.fit_transform(oe.fit_transform(X_red),y_red)
sel.support_

array([False, False, False, False, False,  True, False, False, False,
       False])

In [189]:
print(X_red.columns[sel.support_ == False])

Index(['GoodStudent', 'Age', 'SocioEcon', 'RiskAversion', 'VehicleYear',
       'RuggedAuto', 'MakeModel', 'DrivQuality', 'Mileage'],
      dtype='object')


In [190]:
feature_selection.mutual_info_classif(oe.fit_transform(X_red),y_red)

array([0.        , 0.        , 0.        , 0.        , 0.14641003,
       0.68894937, 0.        , 0.        , 0.        , 0.13133067])

In [200]:
sel = feature_selection.GenericUnivariateSelect(feature_selection.chi2)
X_new = sel.fit_transform(oe.fit_transform(X_red),y_red)
sel.get_support()

array([False, False, False, False, False,  True, False, False, False,
       False])